In [1]:
# Operações para Silver

from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"

print(spark.version)


24/03/18 21:45:42 WARN Utils: Your hostname, desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface enp6s0)
24/03/18 21:45:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/felipe/.ivy2/cache
The jars for the packages stored in: /home/felipe/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a93a4d1-8ff8-4ce4-ab03-9ae36c504d71;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/felipe/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 105ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-9a93a4d1-8ff8-4ce4-ab03-9ae36c504d71
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/3ms)
24/03/18 21:45:42

3.5.0


In [3]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("topicos").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


brbus_table = DeltaTable.create(spark) \
    .tableName("silver_brbus") \
    .addColumn("latitude", dataType="DOUBLE") \
    .addColumn("longitude", dataType="DOUBLE") \
    .addColumn("bus_id", dataType="STRING") \
    .addColumn("city", dataType="STRING") \
    .addColumn("bus_code", dataType="STRING") \
    .addColumn("bus_speed", dataType="DOUBLE") \
    .addColumn("bus_direction", dataType="INT") \
    .addColumn("queried_at", dataType="TIMESTAMP") \
    .addColumn("updated_at", dataType="TIMESTAMP") \
    .location("/home/felipe/code/topicos_dados/lake/silver/silver_brbus") \
    .execute()



24/03/18 21:45:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
spark.sql("DROP TABLE IF EXISTS silver_brbus")

DataFrame[]

In [4]:
from deltalake import DeltaTable

teste = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
teste.show()

+----------+----------+--------------+--------+----------+---------------+-----+----------+-------------------+---------+-------------------+-------------+
|  latitude| longitude|  type_vehicle|bus_code| situation|    situation_2|table|is_adapted|         queried_at|   bus_id|         updated_at|bus_direction|
+----------+----------+--------------+--------+----------+---------------+-----+----------+-------------------+---------+-------------------+-------------+
|-25.439915|-49.271901|MICRO ESPECIAL|     762|  ATRASADO|REALIZANDO ROTA|  2-2|         1|2024-01-30 20:18:21|CWB_JI859|2024-01-30 20:17:00|            1|
|-25.439836|-49.249908|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|    2|         1|2024-01-30 20:18:21|CWB_DN603|2024-01-30 20:18:00|            2|
|-25.470021|-49.212463|MICRO ESPECIAL|     463|NO HORÁRIO|REALIZANDO ROTA|    4|         1|2024-01-30 20:18:21|CWB_DN608|2024-01-30 20:18:00|            1|
|-25.441271| -49.34526|    ARTICULADO|     040|  ATRASADO|REALIZ

In [5]:
from deltalake import DeltaTable

table_brbus = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_brbus/")
table_brbus.show()

+--------+---------+------+----+--------+---------+-------------+----------+----------+
|latitude|longitude|bus_id|city|bus_code|bus_speed|bus_direction|queried_at|updated_at|
+--------+---------+------+----+--------+---------+-------------+----------+----------+
+--------+---------+------+----+--------+---------+-------------+----------+----------+



In [13]:
rj_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_rj/")
rj_silver.schema

StructType([StructField('latitude', DoubleType(), True), StructField('longitude', DoubleType(), True), StructField('bus_speed', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('queried_at', TimestampType(), True), StructField('updated_at', TimestampType(), True), StructField('bus_id', StringType(), True)])

In [17]:
sp_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_sp/")
sp_silver.schema

24/03/18 14:49:10 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=file:/home/felipe/code/topicos_dados/lake/silver/silver_sp/_delta_log, version=0, metadata=Metadata(14e16531-ac45-4749-b2fc-9788aeb6b3fa,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"latitude","type":"double","nullable":true,"metadata":{}},{"name":"longitude","type":"double","nullable":true,"metadata":{}},{"name":"bus_code","type":"string","nullable":true,"metadata":{}},{"name":"station0","type":"string","nullable":true,"metadata":{}},{"name":"station1","type":"string","nullable":true,"metadata":{}},{"name":"queried_at","type":"string","nullable":true,"metadata":{}},{"name":"bus_id","type":"string","nullable":true,"metadata":{}},{"name":"updated_at","type":"string","nullable":true,"metadata":{}}]},List(),Map(),Some(1710783267228)), logSegment=LogSegment(file:/home/felipe/code/topicos_dados/lake/silver/silver_sp/_delta_log,0,WrappedArray(Depr

StructType([StructField('latitude', DoubleType(), True), StructField('longitude', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('station0', StringType(), True), StructField('station1', StringType(), True), StructField('queried_at', TimestampType(), True), StructField('bus_id', StringType(), True), StructField('updated_at', TimestampType(), True)])

In [8]:
cwb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
cwb_silver.schema

StructType([StructField('latitude', DoubleType(), True), StructField('longitude', DoubleType(), True), StructField('type_vehicle', StringType(), True), StructField('bus_code', StringType(), True), StructField('situation', StringType(), True), StructField('situation_2', StringType(), True), StructField('table', StringType(), True), StructField('is_adapted', IntegerType(), True), StructField('queried_at', TimestampType(), True), StructField('bus_id', StringType(), True), StructField('updated_at', TimestampType(), True), StructField('bus_direction', IntegerType(), True)])

In [9]:
bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver.schema

StructType([StructField('longitude', DoubleType(), True), StructField('latitude', DoubleType(), True), StructField('bus_speed', DoubleType(), True), StructField('direction', DoubleType(), True), StructField('bus_code', StringType(), True), StructField('queried_at', TimestampType(), True), StructField('updated_at', TimestampType(), True), StructField('bus_id', StringType(), True), StructField('bus_direction', IntegerType(), True)])

In [4]:
# Adding city values to each DF

from pyspark.sql.functions import lit
# SP

sp_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_sp/")
sp_silver = sp_silver.withColumn("city",lit("Sao Paulo"))
sp_selected = sp_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id","city"])


# RJ
rj_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_rj/")
rj_silver = rj_silver.withColumn("city",lit("Rio de Janeiro"))
rj_selected = rj_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","city"])


# CWB
cwb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_cwb/")
cwb_silver = cwb_silver.withColumn("city",lit("Curitiba"))
cwb_selected = cwb_silver.select(["longitude", "latitude","bus_code","queried_at","updated_at","bus_id","bus_direction","city"])

#BSB

bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver = bsb_silver.withColumn("city",lit("Brasilia"))

bsb_selected = bsb_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","bus_direction","city"])

# Merging into BRBus Table

sp_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
rj_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
cwb_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")
bsb_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")



24/03/18 21:46:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/18 21:46:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/18 21:46:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/03/18 21:46:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/18 21:46:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/18 21:46:04 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/18 21:46:04 WARN MemoryManager: Total allocation exceeds 95,00%

In [4]:
from pyspark.sql.functions import lit
bsb_silver = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_bsb/")
bsb_silver = bsb_silver.withColumn("city",lit("Brasilia"))

bsb_selected = bsb_silver.select(["longitude", "latitude","bus_speed","bus_code","queried_at","updated_at","bus_id","bus_direction","city"])
bsb_selected.write.format("delta").mode("append").save("/home/felipe/code/topicos_dados/lake/silver/silver_brbus")

24/03/18 21:34:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/18 21:34:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/18 21:34:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/18 21:34:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/03/18 21:34:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
24/03/18 21:34:24 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/03/18 21:34:24 WARN MemoryMana

In [5]:
brbus = spark.read.format("delta").load("/home/felipe/code/topicos_dados/lake/silver/silver_brbus/")
brbus.filter(brbus['city']=="Brasilia").sort(brbus['updated_at'],ascending=False).show()
brbus.show()



+----------+----------+----------+--------+--------+---------+-------------+-------------------+-------------------+
|  latitude| longitude|    bus_id|    city|bus_code|bus_speed|bus_direction|         queried_at|         updated_at|
+----------+----------+----------+--------+--------+---------+-------------+-------------------+-------------------+
|-16.008726|-48.082588|BSB_232190|Brasilia|   0.224|     NULL|            2|2024-02-02 23:24:33|2024-02-02 23:24:20|
|-15.913279|-47.757915|BSB_227463|Brasilia|   0.186|     NULL|            2|2024-02-02 23:24:33|2024-02-02 23:24:20|
|-15.914861|-47.769432|BSB_231410|Brasilia|   183.7|     NULL|            1|2024-02-02 23:24:33|2024-02-02 23:24:20|
|-15.756565|-47.772503|BSB_231533|Brasilia|   0.018|     NULL|            2|2024-02-02 23:24:33|2024-02-02 23:24:18|
|-16.024078|-48.031765|BSB_230120|Brasilia|    3312|     NULL|            1|2024-02-02 23:24:33|2024-02-02 23:24:14|
| -15.91022|-47.966846|BSB_230006|Brasilia|   0.224|     NULL|  